ACTUALIZA LAS PROPIEDADES QUE TIENEN MIDIFICACIONES

In [34]:
import ee
ee.Authenticate()

True

In [35]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np

import openpyxl

In [36]:
ee.Initialize()

In [62]:
ruta_catastro='projects/ee-bismarksr17/assets/AZPA_ERROR'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI_02_2026_azpa'

CODIGO_PROPIEDAD='chacra'
NOM_PROPIEDAD='chacra'
CODIGO_CANHERO=''
NOM_CANHERO=''

VARIEDAD='variedad'
ESTADO='RENOVACION'
SUPERFICIE='ha_parcela'

In [63]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

## Eliminar propiedades sobra

In [7]:
# props que presentan irregularidades
lista_eliminar = [106, 622, 1080, 1172, 1302, 1413, 1480, 1540, 1626, 1986,
    1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
    1998, 1999, 2001, 2002, 1421, 1962, 1289, 988, 986, 1446,
    1454, 637, 15, 636, 92, 1735, 880, 328, 19, 201, 114, 437,
    216, 1447, 365, 47, 123, 249, 132, 17, 1491, 1015, 85, 325,
    367, 491, 478, 1081, 744, 752, 313, 616, 1265, 1482, 1743]

In [8]:
df_intersects = gpd.read_file('INTERSECT_2.shp')

DriverError: INTERSECT_2.shp: No such file or directory

In [8]:
df_data = pd.read_excel('DATA_EST.xlsx')

### Eliminar props de DATA_EST.xlsx

In [29]:
df_data = df_data[~df_data['cod_prop'].isin(lista_eliminar)]

In [30]:
df_data.to_excel('data_new.xlsx')

### Eliminar props de INTERSECTS.shp

In [14]:
lista_eliminar = [str(i) for i in lista_eliminar]

In [15]:
df_intersects = df_intersects[~df_intersects['unidad_01'].isin(lista_eliminar)]

In [39]:
path_cat = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - OTROS DATOS\AZPA\SHP\LOTES_AZPA_CANHA.shp'
gdf_cat = gpd.read_file(path_cat)

In [40]:
lista_props = list(set(gdf_cat['chacra']))
print(lista_props)

['SAN JUAN', 'EX ZELLER', 'PIROY', 'YHACA', 'PORTON YOBAI', 'YBYTYMI', 'REVENTON', 'ISLA PUCU', 'ATILANO', 'CAPINARI', 'CORA YBATE', 'SAN JUAN II', 'LOMA PINDO', 'MALVINAS', 'TUPASY PASO', 'SAN BLAS', 'LOMAS', 'BASSEDAN', 'CENTENARIO', 'COLONIA I', 'OVANDO', 'LAS DELICIAS', 'GUAJHO', 'SUR', 'ARECO', 'NORTE', 'ESTACION', 'CASA BLANCA', 'PIRAY', 'YBYTYPE']


In [64]:
lista_props = ['ISLA PUCU']

In [65]:
cods_error

['ISLA PUCU']

In [66]:
len(lista_props)

1

In [43]:
df_intersects = gpd.read_file('INTERSECT_0.shp')

In [67]:
df_intersects.shape

(165112, 16)

In [68]:
cods_error = []

In [69]:
contador = 0

for cod_prop in lista_props:
    print('inicio:', cod_prop)
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_gdf(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    
    try:
        intersect = overlay(lotes_local, vector_local, how="intersection")
    except TypeError as e:
        cods_error.append(cod_prop)
        continue

    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    
    area_diff = (area_02 - area_01)/len(intersect)
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, 'label'], aggfunc=np.sum)
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    
    # Cargar el archivo de Excel existente
    wb = openpyxl.load_workbook('DATA_EST.xlsx')
    # Seleccionar la hoja de trabajo
    ws = wb['data']
    
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3]]
        ws.append(new_row)
    wb.save('DATA_EST.xlsx')
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    #DF.to_file("INTERSECT_0.shp", driver="ESRI Shapefile")
    
    contador = contador + 1 
    print('fin: ', cod_prop)
    print('___________________CONTADOR: ', contador)
    
print('FIN......!')

inicio: ISLA PUCU


C:\Users\Usuario\AppData\Local\Temp\ipykernel_47660\3655867732.py:47: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")


fin:  ISLA PUCU
___________________CONTADOR:  1
FIN......!


C:\Users\Usuario\AppData\Local\Temp\ipykernel_47660\3655867732.py:63: FutureWarning: The provided callable <function sum at 0x000002D7CEA9C1F0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, 'label'], aggfunc=np.sum)


In [70]:
cods_error

[]

In [71]:
df_intersects.tail(5)

,Corte,Fecha cort,MES,area,chacra,codparcela,ha_parcela,idparcela,super,tipo_culti,variedad,zafra,count,label,area_2,geometry
167978,4,01-05-2025,MAYO,2,ISLA PUCU,20,2.252136,41003-20,2.250397,Caña,CTC-4,2025,1,8,0.008401,"POLYGON ((1135660.000 7128805.790, 1135660.000..."
167979,4,01-05-2025,MAYO,2,ISLA PUCU,20,2.252136,41003-20,2.250397,Caña,CTC-4,2025,3,6,0.030820,"POLYGON ((1135670.000 7128740.000, 1135670.000..."
167980,4,01-05-2025,MAYO,2,ISLA PUCU,20,2.252136,41003-20,2.250397,Caña,CTC-4,2025,8,3,0.080820,"POLYGON ((1135670.000 7128790.000, 1135670.000..."
167981,4,01-05-2025,MAYO,2,ISLA PUCU,20,2.252136,41003-20,2.250397,Caña,CTC-4,2025,1,2,0.010820,"POLYGON ((1135670.000 7128780.000, 1135670.000..."
167982,4,01-05-2025,MAYO,2,ISLA PUCU,20,2.252136,41003-20,2.250397,Caña,CTC-4,2025,1,6,0.010820,"POLYGON ((1135670.000 7128800.000, 1135670.000..."


In [72]:
len(df_intersects)

167983

In [74]:
df_intersects.to_file("INTERSECT_01.shp", driver="ESRI Shapefile")

In [73]:
df_intersects['area_2'].sum()

9702.69536185466

In [52]:
df_intersects.head(3)

,Corte,Fecha cort,MES,area,chacra,codparcela,ha_parcela,idparcela,super,tipo_culti,variedad,zafra,count,label,area_2,geometry
0,2,31-12-2025,DICIEMBRE,7,SAN JUAN,14,6.566117,30003-14,6.561181,Caña,CTC-4,2025,14,7,0.128978,"POLYGON ((1146413.871 7132286.213, 1146415.199..."
1,2,31-12-2025,DICIEMBRE,7,SAN JUAN,14,6.566117,30003-14,6.561181,Caña,CTC-4,2025,1,6,0.010487,"POLYGON ((1146440.000 7132250.000, 1146440.000..."
2,2,31-12-2025,DICIEMBRE,7,SAN JUAN,14,6.566117,30003-14,6.561181,Caña,CTC-4,2025,1,5,0.010487,"POLYGON ((1146450.000 7132210.000, 1146450.000..."
